In [1]:
import os
import pvlib
import pandas as pd

In [2]:
YEAR = 1990
STARTDATE = '%d-01-01T00:00:00' % YEAR
ENDDATE = '%d-12-31T23:59:59' % YEAR
TIMES = pd.date_range(start=STARTDATE, end=ENDDATE, freq='H')
INVERTERS = pvlib.pvsystem.retrieve_sam('CECInverter')
INVERTER_10K = INVERTERS['SMA_America__SB10000TL_US__240V_']
CECMODS = pvlib.pvsystem.retrieve_sam('CECMod')
CECMOD_POLY = CECMODS['Canadian_Solar_Inc__CS6X_300P']
CECMOD_MONO = CECMODS['Canadian_Solar_Inc__CS6X_300M']
LATITUDE, LONGITUDE = 40.5137, -108.5449
NREL_API_KEY = os.getenv('NREL_API_KEY', 'DEMO_KEY')
EMAIL = os.getenv('EMAIL', 'bwana.marko@yahoo.com')

In [3]:
header, data = pvlib.iotools.get_psm3(LATITUDE, LONGITUDE, NREL_API_KEY, EMAIL)

In [4]:
# get solar position
data.index = TIMES
sp = pvlib.solarposition.get_solarposition(
        TIMES, LATITUDE, LONGITUDE)

In [5]:
solar_zenith = sp.apparent_zenith.values
solar_azimuth = sp.azimuth.values

In [7]:
dni = data.DNI.values
ghi = data.GHI.values
dhi = data.DHI.values

In [8]:
surface_albedo = data['Surface Albedo'].values
temp_air = data.Temperature.values
dni_extra = pvlib.irradiance.get_extra_radiation(TIMES).values

In [9]:
tracker = pvlib.tracking.singleaxis(solar_zenith, solar_azimuth)

In [10]:
surface_tilt = tracker['surface_tilt']

In [11]:
surface_azimuth = tracker['surface_azimuth']

In [12]:
poa_sky_diffuse = pvlib.irradiance.get_sky_diffuse(
        surface_tilt, surface_azimuth, solar_zenith, solar_azimuth,
        dni, ghi, dhi, dni_extra=dni_extra, model='haydavies')

In [13]:
aoi = tracker['aoi']

In [14]:
poa_ground_diffuse = pvlib.irradiance.get_ground_diffuse(
        surface_tilt, ghi, albedo=surface_albedo)

In [15]:
poa = pvlib.irradiance.poa_components(
        aoi, dni, poa_sky_diffuse, poa_ground_diffuse)

In [16]:
poa_direct = poa['poa_direct']

In [17]:
poa_diffuse = poa['poa_diffuse']

In [18]:
poa_global = poa['poa_global']

In [19]:
iam = pvlib.iam.ashrae(aoi)

In [20]:
effective_irradiance = poa_direct*iam + poa_diffuse

In [21]:
temp_cell = pvlib.temperature.pvsyst_cell(poa_global, temp_air)

In [22]:
# this is the magic
cecparams = pvlib.pvsystem.calcparams_cec(
        effective_irradiance, temp_cell,
        CECMOD_MONO.alpha_sc, CECMOD_MONO.a_ref,
        CECMOD_MONO.I_L_ref, CECMOD_MONO.I_o_ref,
        CECMOD_MONO.R_sh_ref, CECMOD_MONO.R_s, CECMOD_MONO.Adjust)
mpp = pvlib.pvsystem.max_power_point(*cecparams, method='newton')
mpp = pd.DataFrame(mpp, index=TIMES)
Edaily = mpp.p_mp.resample('D').sum()